### Problem 1
#### Question 2 : Hyperparameter search

In [48]:
# USEFUL IMPORTS
import mlp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import sklearn.model_selection
import time
import math

In [2]:
# LOAD THE DATA
data = pd.read_csv("../../mnist_train.csv")
data2 = pd.read_csv("../../mnist_test.csv")

In [30]:
# SPLIT THEM INTO TRAIN VALID TEST SETS
X = data.values[:,1:]/255
Y = data.values[:,0]
X_test = data2.values[:, 1:]/255
Y_test = data2.values[:, 0]
X_train, X_valid, Y_train, Y_valid = sklearn.model_selection.train_test_split( X, Y, test_size=0.1666666, random_state=42)

In [36]:
print("Train set shape ", X_train.shape)
print("Valid set shape ", X_valid.shape)
print("Test  set shape ", X_test.shape)

Train set shape  (50000, 784)
Valid set shape  (10000, 784)
Test  set shape  (10000, 784)


#### Time test

In [47]:
start = time.time()
model = mlp.MLP_2L(784, 784, 480, 10, init="glorot")
model.fit(X_train, Y_train, 1, 1, 0.01, validation_data=(X_valid, Y_valid), epsilon=0.05)
end = time.time()
duration = end - start

days = int(duration/(24 * 60 * 60))
duration = duration % (24 * 60 * 60)
hours = int(duration/( 60 * 60))
duration = duration % (60 * 60)
minutes = int(duration/60)
print("One training epoch with {:d} samples using SGD takes {:d}d {:2d}h {:2d}m".format(X_train.shape[0],days, hours, minutes ))

Input dimension 784	Layer 1 dimension 784	Layer 2 dimension 480	Output dimension 10	 Initilization method glorot	Activation function identity
Total number of parameters : 997050
Train on 50000 samples	Evaluate on 10000
 samples
Epoch 1/1		Total training time 0.0s


KeyboardInterrupt: 

##### Fine tuning the hyperparameters

In [45]:
input_size = 784
output_size = 10
for n1 in [ 500, 784]:
    for n2 in [300, 480]:
        n_parameter = n1 * (input_size + 1) + n2 * (n1 + 1) + output_size * (n2 + 1)
        if n_parameter > 1e6 or n_parameter < 5e5: continue
        model = mlp.MLP_2L(input_size, n1, n2, output_size, init="glorot", verbose=False)
        

In [60]:
duration = 23456


One training epoch with 10 samples using SGD takes 0d  6h 30m
